In [106]:
from __future__ import division
from pyomo.environ import *
from pyomo.core.base.expr import identify_variables
import numpy as np

model = ConcreteModel()


model.n = range(6)

model.x = Var(model.n, within=Binary)

# model.c = np.array([1,2,2,1])
model.c = np.array([1,2,-1,3,-2,1])
# model.a = np.matrix([[-1,-1,0,0],[0,0,-1,-1]])
model.a = np.matrix([[-1,-4,1,0,0,0],[0,0,0,-3,-2,-1]])
# model.b = np.array([-1,-2])
model.b = np.array([-1,-2])
# model.eqmat = np.matrix([1,1,1,1])
model.eqmat = np.matrix([2,-5,1,-2,1,1])
model.eqb = np.array([0])
model.set = range(len(model.b))
consets = [[0],[1]]
# varsets = [[0,1],[2,3]]
varsets = [[0,1,2],[3,4,5]]

In [107]:
def projectBinary(x):
    for row in range(len(x)):
        if x[row] <= 0.5:
            x[row] = 0
        else:
            x[row] = 1
    return x

def projectHyperplane(x,A,b):
    for row in range(A.shape[0]):
        if np.dot(A[row,:],x) - b[row] > 0:
            x = x - ((( (np.dot(np.array(A[row,:]),x))[0] - b[row] )/ ((np.array(np.square(A).sum(axis=1))[row])[0]) ) * np.array(A[row,:].flatten())[0])
    return x

def find_points(A,b):
    """Finds feasible points for a set of linear constraints"""
    points = []
    for i in range(1000):
        counter = 0
        x = np.random.rand(A.shape[1])
        x_old = x+1
        while (np.linalg.norm(x-x_old) > 0.01):
            if counter > 10:
                break
            x_old = x
            x = projectBinary(x)
            x = projectHyperplane(x,A,b)
            counter += 1
        if counter < 10:
            points.append(x)
        
    return np.unique(np.array(points),axis=0)

points = []
for i in range(len(varsets)):
    points.append(find_points(model.a[consets[i],varsets[i]],model.b[consets[i]]))
print(points)

[array([[0., 1., 0.],
       [0., 1., 1.],
       [1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.]]), array([[0., 1., 0.],
       [0., 1., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.]])]


In [94]:
find_points(np.matrix([[2,-5,1,-2,1,1],[-2,5,-1,2,-1,-1],[-1,-4,1,0,0,0],[0,0,0,-3,-2,-1]]),np.array([0,0,-1,-2]))

array([[1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 0., 1., 1.]])

In [45]:
points = [np.array([[0,1,0],[0,1,1],[1,0,0]]),np.array([[0,1,0],[0,1,1]])]

In [24]:
points
points1 = [np.array([[1,1],[0,1]]),np.array([[1,1]])]
print(points1)
points = points1

[array([[1, 1],
       [0, 1]]), array([[1, 1]])]


In [108]:
def create_A_and_rhs(eqmat,eqb,points,varsets):
    big_matrix = np.zeros((len(eqb)+len(points),1))
    for count,set_points in enumerate(points):
        matrix_of_set_i = np.zeros((len(eqb)+len(points),len(set_points)))
        # Now populate the matrix
        matrix_of_set_i[count+1,:] = 1
        for j in range(len(eqb)):
            for column,point in enumerate(set_points):
                matrix_of_set_i[j,column] = np.dot(eqmat[j,varsets[count]],point)[0,0]
        big_matrix = np.concatenate((big_matrix,matrix_of_set_i),axis=1)
    big_matrix = np.delete(big_matrix,0,axis=1)

    rhs = np.concatenate((model.eqb,np.ones(len(points))))
    return big_matrix,rhs

big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
print(big,rhs)

(array([[-5., -4.,  2., -3., -2.,  1.,  2., -2., -1., -1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.]]), array([0., 1., 1.]))


In [109]:
def get_obj_values(c,points,varsets):
    obj_array = np.zeros(1)
    for count,set_points in enumerate(points):
        for point in set_points:
            obj = np.dot(c[varsets[count]],point)
            obj_array = np.append(obj_array,obj)
    obj_array = np.delete(obj_array,0)
    return obj_array

print(get_obj_values(model.c,points,varsets))

[ 2.  1.  1.  3.  2. -2. -1.  3.  4.  1.  2.]


In [110]:
def get_c_A_values(point,setnum,varsets,eqmat,eqb,c):
    obj = np.dot(c[varsets[setnum]],point)
    Acol = np.zeros((len(eqb)+len(varsets),1))
    Acol[setnum+len(eqb),0] = 1
    for i in range(len(eqb)):
        Acol[i,0] = np.dot(eqmat[i,varsets[setnum]],point)[0,0]
    return obj,Acol

In [122]:
rmlp = ConcreteModel()
rmlp.n = range(big.shape[1])
rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
rmlp.con = ConstraintList()
big,rhs = create_A_and_rhs(model.eqmat,model.eqb,points,varsets)
obj = get_obj_values(model.c,points,varsets)
for i in range(big.shape[0]):
    rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

convergenceflag = 0
while convergenceflag == 0:
    convergenceflag = 1
    print ("Duals")
    from pyomo.core import Constraint
    pi = np.zeros(len(model.eqb))
    mu = np.zeros(len(varsets))
    for c in rmlp.component_objects(Constraint, active=True):
        cobject = getattr(rmlp, str(c))
        for index in cobject:
            if index <= len(model.eqb):
                pi[index-1] = rmlp.dual[cobject[index]]
            else:
                mu[index-1-len(pi)] = rmlp.dual[cobject[index]]
            print ("      ", index, rmlp.dual[cobject[index]])
            
    # Now create the subproblem and use the dual values to solve
    for k,varset in enumerate(varsets):
        subprob = ConcreteModel()
        subprob.x = Var(range(len(varset)),within=Binary)
        subprob.con = ConstraintList()
        for set_num,i in enumerate(consets[k]):
            subprob.con.add(expr = 
                           sum(model.a[i,j]*subprob.x[col] for col,j in enumerate(varset)) <= model.b[i]
                           )
        constants = model.c[varsets[k]] - np.dot(pi,model.a[consets[k],varsets[k]])[0,0]
        print(constants)
        print(len(varset))
        subprob.OBJ = Objective(expr = 
                               sum(constants[i]*subprob.x[i] for i in range(len(varset))) - mu[k]
                               )
        solver = SolverFactory('cplex')
        results = solver.solve(subprob, tee = True)
        subprob.solutions.load_from(results)
        if subprob.OBJ() < 0:
            print("Does this print??")
            convergenceflag = 0
            print(subprob.OBJ())
            xval = []
            for v in subprob.component_objects(Var, active=True):
                for index in v:
                    xval.append(v[index].value)
            points[k] = points[k].append(xval)
            newc,newAcol = get_c_A_values(np.array(xval),k,varsets,model.eqmat,model.eqb,model.c)
            obj = np.append(obj,newc)
            big = np.append(big,newAcol,axis=1)
    
    rmlp = ConcreteModel()
    rmlp.n = range(big.shape[1])
    rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
    rmlp.con = ConstraintList()
    for i in range(big.shape[0]):
        rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

    rmlp.OBJ = Objective(expr = 
                        sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

    rmlp.dual = Suffix(direction=Suffix.IMPORT)
    solver = SolverFactory('cplex',solver_io='nl')
    solver.options['presolve'] = 0
    solver.options['lpdisplay'] = 2
    results = solver.solve(rmlp, tee = True)
    rmlp.solutions.load_from(results)
    
for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Initializing dual steep norms . . .
Duals
('      ', 1, 0.0)
('      ', 2, 1.0)
('      ', 3, -2.0)
[ 1.  2. -1.]
3

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpvu3nnc.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpcsya83.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpcsya83.pyomo.lp
Objective sense      : Minimize
Variables            :       4  [Nneg: 1,  Binary: 3]
Objective nonzeros   :       4
Linear constraints   :       2  [Less: 1,  Equal: 1]
  Nonzeros 

In [120]:
rmlp = ConcreteModel()
rmlp.n = range(big.shape[1])
rmlp.lambda_var = Var(rmlp.n,within = NonNegativeReals)
rmlp.con = ConstraintList()
for i in range(big.shape[0]):
    rmlp.con.add(expr = sum( big[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])
rmlp.con.add(expr = rmlp.lambda_var[1] + rmlp.lambda_var[2] <= 0)
# rmlp.con.add(expr = rmlp.lambda_var[1] == 0)
# rmlp.con.add(expr = rmlp.lambda_var[0] == 0)

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Initializing dual steep norms . . .

Iteration      Dual Objective            In Variable           Out Variable
     1               0.000000                     x1               c4 slack
     2               0.000000                     x5                     x2
     3               1.000000                     x7                     x1
('Variable component object', <pyomo.core.base.var.IndexedVar object at 0x000000000A0B7400>)
('   ', 0, 0.0)
('   ', 1, 0.0)
('   ', 2, 0.0)
('   ', 3, 0.0)
('   ', 4, 1.0)
('   ', 5, 0.0)
('   ', 6, 1.0)
('   ', 7, 0.0)
('   ', 8, 0.0)
('   ', 9, 0.0)
('   ', 10, 0.0)
